In [3]:
from httpx import AsyncClient
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
from schemas import VariableEconomica

In [4]:
async with AsyncClient() as client:
    response = await client.get("http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables.asp")

In [5]:
soup = BeautifulSoup(response.read(), 'html.parser')

In [6]:
table = soup.find("table", attrs={"class": "table"})

In [7]:
headers = ["titulo", "fecha", "valor", "link", "serie"]
rows = table.find_all("tr")

In [8]:
table = []
for row in rows:
    if row.text:
        cells = row.find_all("td")
        if len(cells) < 3: continue
        titulo = cells[0].text.strip().lower()
        fecha = datetime.strptime(cells[1].text, "%d/%m/%Y")
        valor = float(cells[2].text.replace('.', '').replace(',', '.'))
        link = f"http://www.bcra.gov.ar" + cells[0].find('a').get('href').strip().replace('\xa0', ' ')
        serie = re.compile(r'serie=(\d{1,6})').search(link).group(1)
        print(link)
        table.append([titulo, fecha, valor, link, serie])
    

http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=246&detalle=Reservas Internacionales del BCRA (en millones de dólares - cifras provisorias sujetas a cambio de valuación)
http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=7927&detalle=Tipo de Cambio Minorista ($ por US$) Comunicación B 9791 - Promedio vendedor
http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=272&detalle=Tipo de Cambio Mayorista ($ por US$) Comunicación A 3500 - Referencia
http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=7935&detalle=Tasa de Política Monetaria (en % n.a.)
http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=7936&detalle=Tasa de Política Monetaria (en % e.a.)
http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=7934&detalle=Tasa fija de precancelación para depósitos con opción de cancelació

In [9]:
df = pd.DataFrame(table, columns=headers)
df

,titulo,fecha,valor,link,serie
0,reservas internacionales del bcra (en millones...,2021-06-07,42292.00,http://www.bcra.gov.ar/PublicacionesEstadistic...,246
1,tipo de cambio minorista ($ por us$) comunicac...,2021-06-09,100.36,http://www.bcra.gov.ar/PublicacionesEstadistic...,7927
2,tipo de cambio mayorista ($ por us$) comunicac...,2021-06-09,95.06,http://www.bcra.gov.ar/PublicacionesEstadistic...,272
3,tasa de política monetaria (en % n.a.),2021-06-09,38.00,http://www.bcra.gov.ar/PublicacionesEstadistic...,7935
4,tasa de política monetaria (en % e.a.),2021-06-09,45.44,http://www.bcra.gov.ar/PublicacionesEstadistic...,7936
5,tasa fija de precancelación para depósitos con...,2021-06-09,30.50,http://www.bcra.gov.ar/PublicacionesEstadistic...,7934
6,tasa mínima para plazos fijos de personas huma...,2021-06-09,37.00,http://www.bcra.gov.ar/PublicacionesEstadistic...,7938


In [10]:
df.loc[0]['link'].replace('\xa0', ' ')

'http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=246&detalle=Reservas Internacionales del BCRA (en millones de dólares - cifras provisorias sujetas a cambio de valuación)'

In [11]:
from schemas import VariableEconomica

In [12]:
list(map(lambda iterrow: VariableEconomica(**dict(iterrow[1])), df.iterrows()))

[VariableEconomica(titulo='reservas internacionales del bcra\xa0(en millones de dólares - cifras provisorias sujetas a cambio de valuación)', link='http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=246&detalle=Reservas Internacionales del BCRA (en millones de dólares - cifras provisorias sujetas a cambio de valuación)', fecha=Timestamp('2021-06-07 00:00:00'), valor=42292.0, serie='246'),
 VariableEconomica(titulo='tipo de cambio minorista ($ por us$) comunicación b 9791 - promedio vendedor', link='http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=7927&detalle=Tipo de Cambio Minorista ($ por US$) Comunicación B 9791 - Promedio vendedor', fecha=Timestamp('2021-06-09 00:00:00'), valor=100.36, serie='7927'),
 VariableEconomica(titulo='tipo de cambio mayorista ($ por us$) comunicación a 3500\xa0- referencia', link='http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp?serie=272&detalle=Tipo 

In [23]:
async with AsyncClient() as client:
    response = await client.post("http://www.bcra.gov.ar/PublicacionesEstadisticas/Principales_variables_datos.asp", data={"fecha_desde": "2021-01-01", "fecha_hasta": "2021-05-05", "primeravez": "1", "serie":"246"})
soup = BeautifulSoup(response.read(), "html.parser")

In [24]:
table = soup.find_all("tbody")
r = [VariableEconomica(fecha=tds[0].text, valor=tds[1].text) for row in table if (tds:=row.find_all('td')) ]

In [25]:
table[0].find_all('td')[1].text.strip().replace('.', '')

'39501'